# Домашнее задание №1

In [1]:
import pandas as pd
import numpy as np

In [105]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]
prices_recommended = [400, 60, 40, 40, 90, 250, 112, 323, 54, 152]
prices_bought = [123, 54, 400, 40]

### Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [73]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #TODO: Ваш код здесь
    flags = np.isin(recommended_list, bought_list)
    recall_atk = float(np.sum(flags[:k])) / len(set(bought_list)) 
    
    return recall_atk

def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)
    
    #TODO: Ваш код здесь

    flags = np.isin(recommended_list, bought_list)
    if sum(flags) == 0:
        return 0
    sum_ = 0
    for i in range(k):
        if flags[i] == True:
            sum_ += prices_recommended[i]
            
    prices_bought_all = np.sum(prices_bought)
    money_recall_atk = sum_ / prices_bought_all

    return money_recall_atk

In [76]:
print(f'𝑅𝑒𝑐𝑎𝑙𝑙@𝐾(𝑖): {recall_at_k(recommended_list, bought_list)}')
print(f'𝑀𝑜𝑛𝑒𝑦𝑅𝑒𝑐𝑎𝑙𝑙@𝐾(𝑖): {money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought)}')

𝑅𝑒𝑐𝑎𝑙𝑙@𝐾(𝑖): 0.5
𝑀𝑜𝑛𝑒𝑦𝑅𝑒𝑐𝑎𝑙𝑙@𝐾(𝑖): 0.713128038897893


### Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MMR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [125]:
def mrr_at_k(recommended_list, bought_list, k=5):
    recommended_list = np.array(recommended_list)
    bought_list = np.array(bought_list)
    recommended_list = recommended_list[:k]
    flags = np.isin(recommended_list, bought_list)
    ranks = np.zeros(k)
    for i in range(k):
        if flags[i]:
            m_rank = np.where(recommended_list == bought_list[i])[0]
            if len(m_rank) > 0:
                ranks[i] = 1 / (m_rank[0] + 1)

    mrr_atk = np.mean(ranks)
    return mrr_atk

In [127]:
print(f'𝑀𝑀𝑅(𝑖)@𝑘: {mrr_at_k(recommended_list, bought_list)}')

𝑀𝑀𝑅(𝑖)@𝑘: 0.05


### Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [133]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #TODO: Ваш код здесь
    
    recommended_list = recommended_list[:k]
    flags = np.isin(recommended_list, bought_list)
    
    #𝐷𝐶𝐺@𝐾(𝑖)
    dcg = np.sum(flags / np.log2(np.arange(2, flags.size + 2)))
    
    #𝐷𝐶𝐺@𝐾(𝑖)
    ideal_flags = np.ones_like(flags)
    idcg = np.sum(ideal_flags / np.log2(np.arange(2, flags.size + 2)))
    
    # 𝑛𝐷𝐶𝐺@𝐾(𝑖)
    ndcg = dcg / idcg
    
    return ndcg

In [134]:
print(f'𝑛𝐷𝐶𝐺@𝐾(𝑖): {ndcg_at_k(recommended_list, bought_list)}')

𝑛𝐷𝐶𝐺@𝐾(𝑖): 0.48522855511632257
